In [7]:
import folium
import pandas as pd

(63969, 15)


In [78]:
# Import data from GeoName
import requests
import json

def getData(place):
    base_url = 'http://api.geonames.org/searchJSON'
    payload = {'username': 'adaisp',
               'country' : 'CH',
               'q' : place,
               'fcodeName' : 'university',
                }
    response = requests.get(base_url, params=payload)
    data = json.loads(response.text)
    try:
        i = 0
        results = {
                'canton': data['geonames'][i]['adminName1'],
                'lat' : data['geonames'][i]['lat'],
                'long' : data['geonames'][i]['lng'],
            }
        while (data['geonames'][i]['adminName1'] == ''):
            i += 1
            results = {
                'canton': data['geonames'][i]['adminName1'],
                'lat' : data['geonames'][i]['lat'],
                'long' : data['geonames'][i]['lng'],
            }
    except:
        results = {
            'canton': 'No Match',
            'lat' : 'No Match',
            'long' : 'No Match',
        }
        
    return results

In [54]:
grant_export = 'P3_GrantExport.csv'
raw_data  = pd.read_csv(grant_export, sep=';', error_bad_lines=False)

uni_data = raw_data.loc[:, ['University', 'Approved Amount']]
uni_data[['Approved Amount']] = uni_data[['Approved Amount']].apply(pd.to_numeric, errors='coerce')

## /!\ Doesn't work yet... we should transform the data type of the column 'approved amount' to integer !
## in order to be able to sum everything and not "concatenate" strings
uni_data = uni_data.groupby(['University'])['Approved Amount'].sum()
uni_data.sort_values(ascending=False, inplace=True)

In [79]:
data = pd.DataFrame(uni_data)
data['Canton'] = ['']*len(data)
data['Latitude'] = ['']*len(data)
data['Longitude'] = ['']*len(data)

for univ,row in data.iterrows():
    try:
        info = getData(univ.split(' - ')[1])
        if (info.get('canton') == 'No Match'):
            info = getData(univ.split(' - ')[0])
    except:
        info = getData(univ.split(' - ')[0])
    data.set_value(univ, 'Canton', info.get('canton'))
    data.set_value(univ, 'Latitude', info.get('lat'))
    data.set_value(univ, 'Longitude', info.get('long'))
    
data.head(100)

,Approved Amount,Canton,Latitude,Longitude
University,,,,
Université de Genève - GE,1.838237e+09,Geneva,46.20222,6.14569
Universität Zürich - ZH,1.826843e+09,Zurich,47.36667,8.55
ETH Zürich - ETHZ,1.635597e+09,Zurich,47.3763,8.54805
Universität Bern - BE,1.519373e+09,Bern,46.94809,7.44744
Universität Basel - BS,1.352251e+09,Basel-City,47.55839,7.57327
Université de Lausanne - LA,1.183291e+09,Zurich,47.36667,8.55
EPF Lausanne - EPFL,1.175316e+09,Vaud,46.51939,6.56673
Université de Fribourg - FR,4.575262e+08,Fribourg,46.80237,7.15128
Université de Neuchâtel - NE,3.832046e+08,Neuchâtel,46.99179,6.931


In [85]:
total_count = len(raw_data)
data['Records'] = raw_data['University'].value_counts()
data

,Approved Amount,Canton,Latitude,Longitude,Records
University,,,,,
Université de Genève - GE,1.838237e+09,Geneva,46.20222,6.14569,6394
Universität Zürich - ZH,1.826843e+09,Zurich,47.36667,8.55,6774
ETH Zürich - ETHZ,1.635597e+09,Zurich,47.3763,8.54805,6153
Universität Bern - BE,1.519373e+09,Bern,46.94809,7.44744,5473
Universität Basel - BS,1.352251e+09,Basel-City,47.55839,7.57327,4746
Université de Lausanne - LA,1.183291e+09,Zurich,47.36667,8.55,4092
EPF Lausanne - EPFL,1.175316e+09,Vaud,46.51939,6.56673,4428
Université de Fribourg - FR,4.575262e+08,Fribourg,46.80237,7.15128,2079
Université de Neuchâtel - NE,3.832046e+08,Neuchâtel,46.99179,6.931,1596


In [28]:
# Creating the map !
cantons_geo = 'ch-cantons.topojson.json'

map = folium.Map(location=[46.73, 8.2], zoom_start=6)

map.choropleth(geo_path=cantons_geo, data=cantons_data,
             columns=['University', 'Approved Amount'],
             key_on='feature.id',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Approved Amount')

/Users/Sym/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: 'threshold_scale' default behavior has changed. Now you get a linear scale between the 'min' and the 'max' of your data. To get former behavior, use folium.utilities.split_six.


TypeError: can't multiply sequence by non-int of type 'float'